In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [ ]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '22'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'

In [ ]:
def cargar_datos(csv_path, table_name):
    conn = None
    cursor = None
    
    try:
        # Conexión con la base de datos
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        # Crear un cursor para ejecutar consultas
        cursor = conn.cursor()

        # Abrir CSV
        with open('../CSV/pedidos.csv', 'r', encoding = 'ISO-8859-1') as file:
            data = file.read().split('\n')
            columns = data[0]
            df = []
            for line in data[1:]:
                df.append(line.split(';'))
        
        # Insertar datos en la tabla
        insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(table_name),
            sql.SQL(', ').join(map(sql.Identifier, columns)),
            sql.SQL(', ').join(map(sql.Placeholder, columns))
        )
        # Iterar sobre las filas del DataFrame y ejecutar la consulta de inserción
        for row in df:
            if not (1 <= row[1] <= 5 and 1 <= row[2] <= 5) and type(row[0]) != int:
                print("Error: invalid values in CSV in row {}".format(row))
            else:
                data = [value for value in row]
                cursor.execute(insert_query, data)
        # Confirmar la transacción
        conn.commit()
        print("Data loaded successfully in table {}.".format(table_name))
    except (Exception, Error) as error:
        print("Error loading data:", error)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    # Ruta al archivo CSV 
    ruta_csv = '../CSV/calificacion.csv'

    # Nombre de la tabla PostgreSQL donde se insertarán los datos
    tabla_destino = 'calificacion'

    cargar_datos_desde_csv(ruta_csv, tabla_destino)